# Protein generator

### Import libraries

In [1]:
import urllib
import pandas as pd

### Function of retrieve a protein family

In [129]:
def get_protein_family(protein_family):
    library = {}
    with urllib.request.urlopen(("https://pfam.xfam.org/family/"+protein_family+"/alignment/uniprot")) as f:
        query = f.read()
    query = str(query, 'utf-8').split("\n")
    for entry in query:
        if (not "#" in entry) and len(entry)>2 :
            uniprot_id, sequence = entry.split()
            library[uniprot_id] = sequence
    return library    

### Function of retrieve conditional tags for a uniprot ID - too slow :-(

In [133]:
def get_conditional_tags(uniprot_id):
    conditional_tags = {}
    with urllib.request.urlopen(("https://www.uniprot.org/uniprot/?query="+str(uniprot_id)+"&sort=score&columns=id,organism,go(molecular%20function),go(cellular%20component),go(biological%20process)&format=tab")) as f:
        query = f.read()
    query = str(query, 'utf-8')
    lines = query.split('\n')
    li_results = [x.split('\t') for x in lines]
    conditions = li_results[0]
    tags = li_results[1]
    for condition,tag in zip(conditions,tags):
        conditional_tags[condition] = tag
    return conditional_tags

In [146]:
family = 'PF00159'
library = get_protein_family(family)

In [147]:
peptide=list(library.keys())
#uniprot_id ="A0A2R8ZHC5.1/98-124"
uniprot_id = peptide

In [155]:
data = {}
for uniprot_id in library:
    tags = get_conditional_tags(uniprot_id)
    data[uniprot_id] = [library[uniprot_id], tags]

10 1060 0.9433962264150944
20 1060 1.8867924528301887
30 1060 2.8301886792452833
40 1060 3.7735849056603774
50 1060 4.716981132075472
60 1060 5.660377358490567
70 1060 6.60377358490566
80 1060 7.547169811320755
90 1060 8.49056603773585
100 1060 9.433962264150944
110 1060 10.377358490566039
120 1060 11.320754716981133
130 1060 12.264150943396226
140 1060 13.20754716981132
150 1060 14.150943396226415
160 1060 15.09433962264151
170 1060 16.037735849056602
180 1060 16.9811320754717
190 1060 17.92452830188679
200 1060 18.867924528301888
210 1060 19.81132075471698
220 1060 20.754716981132077
230 1060 21.69811320754717
240 1060 22.641509433962266
250 1060 23.58490566037736
260 1060 24.528301886792452
270 1060 25.471698113207548
280 1060 26.41509433962264
290 1060 27.358490566037734
300 1060 28.30188679245283
310 1060 29.245283018867923
320 1060 30.18867924528302
330 1060 31.132075471698112
340 1060 32.075471698113205
350 1060 33.0188679245283
360 1060 33.9622641509434
370 1060 34.905660377358

In [194]:
peptide=list(data.keys())[218]

In [195]:
data[peptide], peptide

(['PPKPENPG..EDAPPEELAKYYTALRHYINLITRQRY',
  {'Entry': 'A0A3B4FZG0',
   'Organism': 'Pundamilia nyererei',
   'Gene ontology (molecular function)': 'hormone activity [GO:0005179]; type 2 neuropeptide Y receptor binding [GO:0031843]',
   'Gene ontology (cellular component)': 'extracellular region [GO:0005576]',
   'Gene ontology (biological process)': ''}],
 'A0A3B4FZG0.1/30-64')